Purpose:  Build framework to compare raw data vs. extracted for use in COVID-19 R&D Dashboard on a reoccurring basis.  Identify any lost/mistranslated data, be able to fix errors/bugs in back-end code as they arise.
    
Raw data source: https://docs.google.com/spreadsheets/d/11FlafRMeQ2D6doEX_CMHyW4OqnXkp1FfrkLdsxhd0do/edit#gid=1988095192
Extracted data source:  https://c19-vac-rnd-dash-api.herokuapp.com/assets

Process - 1) read/describe raw data (downloaded as CSV file) - note link above is from coviddash.org link, "Full Dataset Here," last updated 6/16/20.  Downloaded CSV from this dataset
            2) read/describe extracted data (JSON), read directly from link above
            3) clean raw to compare to extracted data - remove extraneous rows and columns, ensure same # of unique products (i.e. one row per product)
            4) identify matching fields between the two
            5) compare/join/merge like fields, identify any differences in data set between the two
            
To-do - if this framework is suitable, then I think this comparison can be done directly from the raw data source file online, w/o need to download first (I believe this requires API credentials and some other configuration)

For time being, download CSV from first link above and change name and pathway for files in read commands below, install/import libraries as necessary

In [2]:
#import libraries
import numpy as np
import pandas as pd
import dask.dataframe as dd
import datacompy
import tabulate
import beautifultable as bt
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as st
import json
%matplotlib inline

In [8]:
#import product transform modules

import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/_init_.py'
import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_geolocation.py'
import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_milestone_transforms.py'
import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_renaming.py'
import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_sponsor_transforms.py'
import 'https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_transforms.py'    

SyntaxError: invalid syntax (<ipython-input-8-069058a4fd96>, line 3)

In [90]:
#Read raw data (CSV) (change read pathway as necessary)
df1=pd.read_csv('https://github.com/albrightjustinm/py-api-vac-rnd-dash/Dataset_V1.2_(06-30-20).csv',skiprows=2)
pd.set_option('max_columns', None)
pd.set_option('max_rows', 10)
df1.head(9999)

#notes- two rows for each product(yes and no source), 49 columns altogether

,ID,Source?,Product Name - Preferred,Product Name - Chemical,Product Name - Brand,Sponsor,Intervention Type,Indication,Molecule Type,Therapeutic Approach,New/Repurposed,Funding/Manufacturing/Research/Other Partners,Country,Status,Notes,Unnamed: 15,Current Stage,Unnamed: 17,Discovery Started,Pre-Clinical Studies Started,Lead Selection Finalized,Clinical Batch Finalized,IND or Equivalent Approval Finalized,Phase 1 Started,Phase 2 Started,Phase 3 Started,NDA or equivalent Approval Finalized,Unnamed: 27,Unnamed: 28,Phase,Condition or Disease,Number of Participants,Accepts Healthy Subjects,# of Sites,Sites Locations,Study Start Date,Primary Completion DAte,Study Completion Date,Registry Link,How to participate,Unnamed: 40,Data Entry 1 Owner,Date Entry 1 Performed,Data Entry 2 Owner,Date Entry 2 Performed,Data Entry Update Owner,Date Update Performed,Unnamed: 47,Last Updated
0,1,No,mRNA-1273,NaN,mRNA-1273,Moderna; National Institute of Allergy and Inf...,Vaccine - Prophylactic,COVID-19,Nucleic acid based therapies/vaccines,Vaccine,New,CEPI; Lonza; BARDA,United States,Ongoing,mRNA-based vaccine,NaN,Phase 1,NaN,1/11/2020,SKIPPED,1/13/2020,2/7/2020,3/4/2020,3/16/2020,NaN,NaN,NaN,NaN,NaN,2,COVID-19,600,Yes,10,Meridian Clinical Research - Savannah - Georgi...,5/25/2020,3/21/2020,8/21/2020,NCT04405076,KPWA.vaccine@kp.org,NaN,Mats,3/27/2020,Matthew,5/31/2020,Priya Kaur,6/18/2020,NaN,6/18/2020
1,1,Yes,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,NaN,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.modernatx.com/modernas-work-potent...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,NaN,https://www.nih.gov/news-events/news-releases/...,https://www.nih.gov/news-events/news-releases/...,https://www.modernatx.com/modernas-work-potent...,https://www.modernatx.com/modernas-work-potent...,https://www.modernatx.com/modernas-work-potent...,https://www.modernatx.com/modernas-work-potent...,https://www.modernatx.com/modernas-work-potent...,https://www.modernatx.com/modernas-work-potent...,NaN,NaN,NaN,NaN,NaN,https://investors.modernatx.com/news-releases/...,https://investors.modernatx.com/news-releases/...,NaN,https://investors.modernatx.com/news-releases/...,NaN,NaN,https://investors.modernatx.com/news-releases/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,No,Novavax Vaccine,NVX-CoV2373,Novavax Vaccine,Novavax Inc.; Emergent BioSolutions Inc.,Vaccine - Prophylactic,COVID-19,Subunit Vaccines,Other,New,CEPI,United States,Ongoing,not totally clear that it is prophylactic vacc...,NaN,Phase 1,NaN,NaN,3/10/2020,NaN,NaN,NaN,5/25/2020,NaN,NaN,NaN,NaN,NaN,1/2,COVID-19,131,Yes,2,Herston - Australia; Melbourne - Australia,5/25/2020,12/31/2020,7/31/2020,NCT04368988,B.Georgievska@nucleusnetwork.com.au,NaN,James,4/2/2020,Matthew,5/31/2020,NaN,NaN,NaN,5/31/2020
3,2,Yes,https://investors.emergentbiosolutions.com/new...,http://ir.novavax.com/news-releases/news-relea...,http://ir.novavax.com/news-releases/news-relea...,https://investors.emergentbiosolutions.com/new...,https://investors.emergentbiosolutions.com/new...,https://ir.novavax.com/news-releases/news-rele...,https://ir.novavax.com/news-releases/news-rele...,NaN,https://ir.novavax.com/news-releases/news-rele...,https://ir.novavax.com/news-releases/news-rele...,https://ir.novavax.com/news-releases/news-rele...,NaN,https://clinicaltrials.gov/ct2/show/NCT0436898...,NaN,https://clinicaltrials.gov/ct2/show/NCT0436898...,NaN,NaN,https://ir.novavax.com/news-releases/news-rele...,NaN,NaN,NaN,https://clinicaltrials.gov/ct2/show/NCT0436898...,NaN,NaN,NaN,NaN,NaN,https://clinicaltrials.gov/ct2/sh

In [19]:
#save column names
df1.columns

Index(['ID', 'Source?', 'Product Name - Preferred', 'Product Name - Chemical',
       'Product Name - Brand', 'Sponsor', 'Intervention Type', 'Indication',
       'Molecule Type', 'Therapeutic Approach', 'New/Repurposed',
       'Funding/Manufacturing/Research/Other Partners', 'Country', 'Status',
       'Notes', 'Unnamed: 15', 'Current Stage', 'Unnamed: 17',
       'Discovery Started', 'Pre-Clinical Studies Started',
       'Lead Selection Finalized', 'Clinical Batch Finalized',
       'IND or Equivalent Approval Finalized', 'Phase 1 Started',
       'Phase 2 Started', 'Phase 3 Started',
       'NDA or equivalent Approval Finalized', 'Unnamed: 27', 'Unnamed: 28',
       'Phase', 'Condition or Disease', 'Number of Participants',
       'Accepts Healthy Subjects', '# of Sites', 'Sites Locations',
       'Study Start Date', 'Primary Completion DAte', 'Study Completion Date',
       'Registry Link', 'How to participate', 'Unnamed: 40',
       'Data Entry 1 Owner', 'Date Entry 1 Performed'

In [84]:
#Read extracted data (JSON)
df2=pd.read_json('https://c19-vac-rnd-dash-api.herokuapp.com/assets', orient=None, typ='frame', dtype=True, convert_axes=True, convert_dates=True, keep_default_dates=True, numpy=False, precise_float=False, date_unit=None, encoding=None, lines=False, chunksize=None, compression='infer')
df2=df2.sort_values(by='productId')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
#df2=df2.append({'productId':""}, ignore_index=True)
df2.head(9999)

,acceptsHealthySubjects,brandName,chemicalName,conditionOrDisease,countries,countryCodes,currentStage,indication,interventionType,milestones,moleculeType,notes,numSites,otherPartners,phase,preferredName,primaryCompletionDate,productId,repurposed,siteLocations,sources,sponsors,status,studyCompletionDate,studyStartDate,therapeuticApproach,trialId
0,,mRNA-1273,,COVID-19,[United States],[USA],Phase 1,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': None, 'm...",Nucleic acid based therapies/vaccines,mRNA-based vaccine,2,CEPI,1,mRNA-1273,"Tue, 01 Jun 2021 00:00:00 GMT",1,New,"[{'city': 'Seattle', 'country': 'USA', 'lat': ...",[https://www.nih.gov/news-events/news-releases...,[{'sponsorId': '375ad6c4b12a03acefcf5e9b052423...,Ongoing,"Tue, 01 Jun 2021 00:00:00 GMT","Mon, 16 Mar 2020 00:00:00 GMT",Vaccine,NCT04283461
1,,Novavax Vaccine,,,[United States],[USA],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Tue, 10...",Subunit Vaccines,not totally clear that it is prophylactic vacc...,,CEPI,,Novavax Vaccine,,2,New,[],[https://investors.emergentbiosolutions.com/ne...,[{'sponsorId': '605f6647f1bdd3849bac0626225a6e...,Ongoing,,,Other,
2,,BNT-162,,,[Germany],[DEU],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Mon, 16...",Nucleic acid based therapies/vaccines,Expect clinical testing in late april. Note th...,,Polymun,,BNT162,,3,New,[],[https://www.pfizer.com/news/press-release/pre...,[{'sponsorId': '7c4d34cd18acefd9b97f8b918fe356...,Ongoing,,,Other,
3,,Imperial College London Vaccine,,,[],[],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Mon, 10...",Nucleic acid based therapies/vaccines,Expect clinical testing in summer,,,,Imperial College London Vaccine,,4,New,[],[https://www.imperial.ac.uk/news/196313/in-pic...,[{'sponsorId': '1a693dd5acf9f3ae07a65241da0b2f...,Ongoing,,,Other,
4,,CELLECTRA®,INO-4800,COVID-19,[United States],[USA],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': None, 'm...",Nucleic acid based therapies/vaccines,,,"US DoD,Bill and Melinda Gates Foundation,Coali...",1,INO-4800,,5,New,"[{'city': None, 'country': 'USA', 'lat': 37.09...",[https://www.precisionvaccinations.com/vaccine...,[{'sponsorId': 'a28cb988e230163b5e185750856269...,Ongoing,,,Monoclonal antibodies,
5,,Ad26 SARS-CoV-2,,,[Belgium],[BEL],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Tue, 31...",Gene therapies,Expect clinical testing no later than fall 2020,,BARDA,,Ad26 SARS-CoV-2,,6,New,[],[https://www.hhs.gov/about/news/2020/03/30/hhs...,[{'sponsorId': '79c2245b2e99e2ee02f6cc48aaefe0...,Ongoing,,,Other,
6,,Sanofi Vaccine,,,[France],[FRA],Pre-Clinical Testing,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Tue, 18...",Subunit Vaccines,they are repurposing vaccines that have been p...,,BARDA,,Sanofi Vaccine,,7,Repurposed-Not Approved,[],[http://www.news.sanofi.us/2020-02-18-Sanofi-j...,[{'sponsorId': '551135a9f6ade78853bbd5fdbd6628...,Ongoing,,,Other,
7,,Regeneron Therapeutic,,,[United States],[USA],Discovery,COVID-19,Therapeutic,"[{'category': 'pre-clinical', 'date': 'Tue, 04...",Protein-based (including monoclonal antibodies),Plan to start human trials in summer 2020. Tec...,,,,Regeneron Therapeutic,,8,New,[],[https://investor.regeneron.com/index.php/news...,[{'sponsorId': 'a7cbf979a98fdc448b32b211e3d5e8...,Ongoing,,,Monoclonal antibodies,
8,,Vaxart Vaccine,,,[United States],[USA],Discovery,COVID-19,Vaccine - Prophylactic,"[{'category': 'pre-clinical', 'date': 'Wed, 18...",Subunit Vaccines,Oral tablet. Recombinant vaccine. Plan to init...,,Emergent,,Vaxart Vaccine,,9,New,[],[https://investors.vaxart.com/news-releases/ne...,[{'sponsorId': '80a6767f041a7aef5269023e2f54d9...,Ongoing,,,,
9,,University of Queensland Vaccine,,,[Australia],[AUS],Discovery,COVID-19,Vaccine - Prophylactic,"[{'categor

In [97]:
#One extra or duplicate row in JSON (565) vs. CSV (564)....what is easiest way to identify this for comparison below?

#duplicateRowsDF = df2[df2.duplicated(['sources'])]
#print(duplicateRowsDF)

In [11]:
#check format of milestones column
df2.milestones[0]

[{'category': 'pre-clinical',
  'date': None,
  'milestoneId': 12,
  'name': 'pre_clinical_studies',
  'status': 'SKIPPED'},
 {'category': 'pre-clinical',
  'date': 'Mon, 13 Jan 2020 00:00:00 GMT',
  'milestoneId': 13,
  'name': 'lead_selection',
  'status': 'COMPLETED'},
 {'category': 'manufacturing',
  'date': 'Fri, 07 Feb 2020 00:00:00 GMT',
  'milestoneId': 21,
  'name': 'clinical_batch',
  'status': 'COMPLETED'},
 {'category': 'regulatory',
  'date': 'Wed, 04 Mar 2020 00:00:00 GMT',
  'milestoneId': 31,
  'name': 'ind',
  'status': 'COMPLETED'},
 {'category': 'clinical_development',
  'date': 'Mon, 16 Mar 2020 00:00:00 GMT',
  'milestoneId': 41,
  'name': 'phase_1',
  'status': 'ONGOING'},
 {'category': 'pre-clinical',
  'date': 'Sat, 11 Jan 2020 00:00:00 GMT',
  'milestoneId': 11,
  'name': 'discovery',
  'status': 'COMPLETED'}]

In [27]:
#save column names
df2.columns

Index(['acceptsHealthySubjects', 'brandName', 'chemicalName',
       'conditionOrDisease', 'countries', 'countryCodes', 'currentStage',
       'indication', 'interventionType', 'milestones', 'moleculeType', 'notes',
       'numSites', 'otherPartners', 'phase', 'preferredName',
       'primaryCompletionDate', 'productId', 'repurposed', 'siteLocations',
       'sources', 'sponsors', 'status', 'studyCompletionDate',
       'studyStartDate', 'therapeuticApproach', 'trialId'],
      dtype='object')

In [16]:
#Filter raw data so it has same number of rows as extracted (i.e. only 1 row per product)

df1filtA=df1[df1["Source?"]=="No"]
print(df1filtA)

#note - still some blank rows at bottom...can this be filtered by 1 command or need to be separated (as done below)?

       ID Source?         Product Name - Preferred Product Name - Chemical  \
0       1      No                        mRNA-1273                     NaN   
2       2      No                  Novavax Vaccine             NVX-CoV2373   
4       3      No                           BNT162                     NaN   
6       4      No  Imperial College London Vaccine                     NaN   
8       5      No                         INO-4800                INO-4800   
...   ...     ...                              ...                     ...   
1198  600      No                              NaN                     NaN   
1200  601      No                              NaN                     NaN   
1202  602      No                              NaN                     NaN   
1204  603      No                              NaN                     NaN   
1206  604      No                              NaN                     NaN   

                 Product Name - Brand  \
0                     

In [17]:
#filter blank rows at bottom
df1filtB=df1filtA[df1filtA["Sponsor"].notnull()]

print(df1filtB)

       ID Source?         Product Name - Preferred Product Name - Chemical  \
0       1      No                        mRNA-1273                     NaN   
2       2      No                  Novavax Vaccine             NVX-CoV2373   
4       3      No                           BNT162                     NaN   
6       4      No  Imperial College London Vaccine                     NaN   
8       5      No                         INO-4800                INO-4800   
...   ...     ...                              ...                     ...   
1118  560      No                              NaN                     NaN   
1120  561      No                              NaN                     NaN   
1122  562      No                              NaN                     NaN   
1124  563      No                              NaN                     NaN   
1126  564      No                              NaN                     NaN   

                 Product Name - Brand  \
0                     

In [43]:
#filter all NaN cells to blank
df1filtC=df1filtB.fillna('')
print(df1filtC)

       ID Source?         Product Name - Preferred Product Name - Chemical  \
0       1      No                        mRNA-1273                           
2       2      No                  Novavax Vaccine             NVX-CoV2373   
4       3      No                           BNT162                           
6       4      No  Imperial College London Vaccine                           
8       5      No                         INO-4800                INO-4800   
...   ...     ...                              ...                     ...   
1118  560      No                                                            
1120  561      No                                                            
1122  562      No                                                            
1124  563      No                                                            
1126  564      No                                                            

                 Product Name - Brand  \
0                     

In [103]:
#change column names to match JSON column names
# code below copied from https://github.com/albrightjustinm/py-api-vac-rnd-dash/blob/master/api/utils/transform/products/product_renaming.py
# added pound signs where not congruent with current data sets, or raw fields reformatted to milestones in extracted

df1filtD = (df1filtC.rename(columns={
    'ID': 'productId', #'product_id',#
    'Source?': 'sources', #'source',
    'Product Name - Preferred': 'preferredName', #'preferred_name',
    'Product Name - Chemical': 'chemicalName', #'chemical_name',
    'Product Name - Brand': 'brandName', #'brand_name',
    'Sponsor': 'sponsors',
    'Intervention Type': 'interventionType', #'intervention_type',
    'Indication': 'indication',
    'Molecule Type': 'moleculeType', #'molecule_type',
    'Therapeutic Approach': 'therapeuticApproach', #'therapeutic_approach',
    'New/Repurposed': 'repurposed',
    'Notes': 'notes',
    'Funding/Manufacturing/Research/Other Partners': 'otherPartners', #'other_partners',
    'Country': 'countries',
    'Current Stage': 'currentStage', #'current_stage',
    #milestones - 'Pre-Clinical Studies Started': 'pre_clinical_studies_started_date',
    #milestones - 'Lead Selection Finalized': 'lead_selection_finalized_date',
    #milestones - 'Clinical Batch Finalized': 'clinical_batch_finalized_date',
    #milestones - 'IND or Equivalent Approval Finalized': 'ind_finalized_date',
    #milestones - 'Phase 1 Started': 'phase_1_started_date',
    #milestones - 'Phase 2 Started': 'phase_2_started_date',
    #milestones - 'Phase 3 Started': 'phase_3_started_date',
    #milestones - 'NDA or equivalent Approval Finalized': 'nda_finalized',
    'Phase': 'phase',
    'Condition or Disease': 'conditionOrDisease', #'condition_or_disease',
    # field not in extracted data - 'Number of Participants': 'number_participants',
    'Accepts Healthy Subjects': 'acceptsHealthySubjects', #'accepts_healthy_subjects',
    '# of Sites': 'numSites', #'num_sites',
    'Sites Locations': 'siteLocations', #'site_locations',
    'Study Start Date': 'studyStartDate', #'study_start_date',
    'Primary Completion DAte': 'primaryCompletionDate', #'primary_completion_date',
    'Study Completion Date': 'studyCompletionDate', #'study_completion_date',
    # field not in extracted data - 'How to participate': 'participation_link',
    # milestones - 'Discovery Started': 'discovery_started_date',
    # field not in raw data:  'CTG Identifier': 'trial_id',
    'Status': 'status'}))
print(df1filtD)

      productId sources                    preferredName chemicalName  \
0             1      No                        mRNA-1273                
2             2      No                  Novavax Vaccine  NVX-CoV2373   
4             3      No                           BNT162                
6             4      No  Imperial College London Vaccine                
8             5      No                         INO-4800     INO-4800   
...         ...     ...                              ...          ...   
1118        560      No                                                 
1120        561      No                                                 
1122        562      No                                                 
1124        563      No                                                 
1126        564      No                                                 

                            brandName  \
0                           mRNA-1273   
2                     Novavax Vaccine   


In [104]:
#check and store names
df1filtD.columns

Index(['productId', 'sources', 'preferredName', 'chemicalName', 'brandName',
       'sponsors', 'interventionType', 'indication', 'moleculeType',
       'therapeuticApproach', 'repurposed', 'otherPartners', 'countries',
       'status', 'notes', 'Unnamed: 15', 'currentStage', 'Unnamed: 17',
       'Discovery Started', 'Pre-Clinical Studies Started',
       'Lead Selection Finalized', 'Clinical Batch Finalized',
       'IND or Equivalent Approval Finalized', 'Phase 1 Started',
       'Phase 2 Started', 'Phase 3 Started',
       'NDA or equivalent Approval Finalized', 'Unnamed: 27', 'Unnamed: 28',
       'phase', 'conditionOrDisease', 'Number of Participants',
       'acceptsHealthySubjects', 'numSites', 'siteLocations', 'studyStartDate',
       'primaryCompletionDate', 'studyCompletionDate', 'Registry Link',
       'How to participate', 'Unnamed: 40', 'Data Entry 1 Owner',
       'Date Entry 1 Performed', 'Data Entry 2 Owner',
       'Date Entry 2 Performed', 'Data Entry Update Owner',


In [105]:
df1filtD.head()

,productId,sources,preferredName,chemicalName,brandName,sponsors,interventionType,indication,moleculeType,therapeuticApproach,repurposed,otherPartners,countries,status,notes,Unnamed: 15,currentStage,Unnamed: 17,Discovery Started,Pre-Clinical Studies Started,Lead Selection Finalized,Clinical Batch Finalized,IND or Equivalent Approval Finalized,Phase 1 Started,Phase 2 Started,Phase 3 Started,NDA or equivalent Approval Finalized,Unnamed: 27,Unnamed: 28,phase,conditionOrDisease,Number of Participants,acceptsHealthySubjects,numSites,siteLocations,studyStartDate,primaryCompletionDate,studyCompletionDate,Registry Link,How to participate,Unnamed: 40,Data Entry 1 Owner,Date Entry 1 Performed,Data Entry 2 Owner,Date Entry 2 Performed,Data Entry Update Owner,Date Update Performed,Unnamed: 47,Last Updated
0,1,No,mRNA-1273,,mRNA-1273,Moderna; National Institute of Allergy and Inf...,Vaccine - Prophylactic,COVID-19,Nucleic acid based therapies/vaccines,Vaccine,New,CEPI; Lonza; BARDA,United States,Ongoing,mRNA-based vaccine,,Phase 1,,1/11/2020,SKIPPED,1/13/2020,2/7/2020,3/4/2020,3/16/2020,,,,,,2,COVID-19,600,Yes,10,Meridian Clinical Research - Savannah - Georgi...,5/25/2020,3/21/2020,8/21/2020,NCT04405076,KPWA.vaccine@kp.org,,Mats,3/27/2020,Matthew,5/31/2020,Priya Kaur,6/18/2020,,6/18/2020
2,2,No,Novavax Vaccine,NVX-CoV2373,Novavax Vaccine,Novavax Inc.; Emergent BioSolutions Inc.,Vaccine - Prophylactic,COVID-19,Subunit Vaccines,Other,New,CEPI,United States,Ongoing,not totally clear that it is prophylactic vacc...,,Phase 1,,,3/10/2020,,,,5/25/2020,,,,,,1/2,COVID-19,131,Yes,2,Herston - Australia; Melbourne - Australia,5/25/2020,12/31/2020,7/31/2020,NCT04368988,B.Georgievska@nucleusnetwork.com.au,,James,4/2/2020,Matthew,5/31/2020,,,,5/31/2020
4,3,No,BNT162,,BNT-162,Pfizer Inc.; BioNTech SE,Vaccine - Prophylactic,COVID-19,Nucleic acid based therapies/vaccines,Other,New,Polymun,Germany,Ongoing,Expect clinical testing in late april. Note th...,,Phase 1,,,3/16/2020,,,,4/23/2020,,,,,,1/2,Respiratory Infections,200,Yes,1,Berlin - Germany,04/23/2020,8/1/2020,8/1/2020,NCT04380701,,,James,4/2/2020,Matthew,5/31/2020,,,,5/31/2020
6,4,No,Imperial College London Vaccine,,Imperial College London Vaccine,Imperial College London; Maravai Lifesciences ...,Vaccine - Prophylactic,COVID-19,Nucleic acid based therapies/vaccines,Other,New,,United Kingdon,Ongoing,Expect clinical testing in summer,,Pre-Clinical Testing,,,2/10/2020,,,,,,,,,,,,,,,,,,,,,,James,4/2/2020,Matthew,5/31/2020,,,,5/31/2020
8,5,No,INO-4800,INO-4800,CELLECTRA®,Inovio Pharmaceuticals Inc.; Beijing Advaccine...,Vaccine - Prophylactic,COVID-19,Nucleic acid based therapies/vaccines,Monoclonal antibodies,New,US DoD; Bill and Melinda Gates Foundation; Coa...,United States,Ongoing,,,Phase 1,,1/23/2020,,,3/24/2020,,4/3/2020,,,,,,1,COVID-19,120,Yes,3,Central Kentucky Research Associates - Lexingt...,4/3/2020,7/21,7/21,NCT04336410,clinical.trials@inovio.com,,Joseph Malinao,4/2/2020,Matthew,5/31/2020,Priya Kaur,6/18/2020,,6/18/2020


In [106]:
#Alternative to comparing dataframes w/o merging
compare=datacompy.Compare(df1filtD,
                          df2,
                          join_columns='productId',
                          abs_tol=0, rel_tol=0,     
                          df1_name='Raw CSV',
                          df2_name='JSON',
                               ignore_spaces=True, 
                               ignore_case=True)

compare.matches(ignore_extra_columns=True)
print(compare.report())
print(compare.intersect_columns())

#To-do:
    #1.  ignore puncuation(semicolon vs. comma, extra spaces, brackets)
    #2.  how to compare "milestone" data to combined fields in JSON
    #3.  show all values not matching, not just a sample
    #4.  convert to table or exportable files once finished?

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0   Raw CSV       49   564
1      JSON       27   565

Column Summary
--------------

Number of columns in common: 24
Number of columns in Raw CSV but not in JSON: 25
Number of columns in JSON but not in Raw CSV: 3

Row Summary
-----------

Matched on: productid
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 564
Number of rows in Raw CSV but not in JSON: 0
Number of rows in JSON but not in Raw CSV: 1

Number of rows with some compared columns unequal: 564
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 23
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 7,066

Columns with Unequal Values or Types
------------------------------------

                    Column Raw CSV dtype JSON dtype

Ununsed comparison/merge code below - for reference only

In [102]:
#Compare columns between dataframes, identify data that's not equal
#df1filtD.where(df1filtD.values==df2.values).notna('brandname')

#ValueError: Array conditional must be same shape as self

In [28]:
#merge dataframes on unique ID#
#merged = df1filtA.merge(df2,how='outer',left_on=['ID'],right_on=["productId"])
#pd.set_option('max_columns', None)
#merged.head()

In [186]:
#don't use for now
#merged['brandName'].equals(merged['Product Name - Brand'])

#E = np.where(merged["brandName"] == merged["Product Name - Brand"], ".", 'OFF')
#D = np.where(merged["chemicalName"] == merged["Product Name - Chemical"], ".", 'OFF')
#print(E,D)
#table=[[E],[D]]
#print(tabulate(table))